# Setup

In [99]:
# !pip install -U git+https://github.com/storemesh/mapping-v2.git

In [6]:
import dsm_mapping
import pandas as pd

mapping = dsm_mapping.mapping.Mapping(
    services_uri='http://192.168.24.170:8180',
    api_key="KEL1zxVp.t3EAe1C1oWmSJh3cwKnG3uHeOXESiqKq",
    project_id=7
)

{'id': 7, 'name': 'Country', 'type': 'nGram'}


## import master data

In [11]:
df_master = pd.read_csv('master_country.csv')

In [22]:
df=df_master[['id','country_en','country_th']]
df2=df.melt(id_vars=['id'],value_vars=['country_en','country_th'])
df2

,id,variable,value
0,2023AFG,country_en,Afghanistan
1,2023ALB,country_en,Albania
2,2023ATA,country_en,Antarctica
3,2023DZA,country_en,Algeria
4,2023ASM,country_en,American Samoa
...,...,...,...
495,2023WSM,country_th,ซามัว
496,2023YEM,country_th,เยเมน
497,2023ZMB,country_th,แซมเบีย
498,2023001,country_th,โลก


In [23]:
mapping.bulk_master_data(df=df2, column_id='id', column_text='value')

{'msg': 'master data created'}

## mapping

In [10]:
df_query=pd.read_csv('country_person.csv')

In [26]:
df_query['result']=df_query['country'].apply(mapping.search)

In [28]:
df_query

,country,result
0,Thailand,"Success({'id': 711, 'master_id': '2023THA', 't..."
1,Poland,"Success({'id': 668, 'master_id': '2023POL', 't..."
2,Singapore,"Success({'id': 693, 'master_id': '2023SGP', 't..."
3,Switzerland,"Success({'id': 708, 'master_id': '2023CHE', 't..."
4,Japan,"Success({'id': 603, 'master_id': '2023JPN', 't..."
...,...,...
332,South korea,"Success({'id': 701, 'master_id': '2023SSD', 't..."
333,United state,"Success({'id': 732, 'master_id': '2023VIR', 't..."
334,Netherland,"Success({'id': 642, 'master_id': '2023NLD', 't..."
335,U.s.a.,Error(masterdata not found)


In [79]:
df_query['value']  = df_query['result'].apply(lambda x: x.value.get('text') if x.value else '')

In [80]:
df_query['error']  = df_query['result'].apply(lambda x: x.error)

In [87]:
pd.set_option('display.max_rows', None) 
df3=df_query[['country','value','error']]
df3

,country,value,error
0,Thailand,Thailand,None
1,Poland,Poland,None
2,Singapore,Singapore,None
3,Switzerland,Switzerland,None
4,Japan,Japan,None
5,India,India,None
6,Kenya,Kenya,None
7,United,United Arab Emirates,None
8,Egypt,Egypt,None
9,Bangladesh,Bangladesh,None


## Dask

In [95]:
import dask.dataframe as dd
df_query=pd.read_csv('country_person.csv')
ddf = dd.from_pandas(df_query, npartitions=16)

In [96]:
ddf['result'] = ddf['country'].apply(mapping.search)

/home/wasit/miniconda3/envs/dsmde/lib/python3.10/site-packages/dask/dataframe/core.py:4522: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('country', 'object'))

  warnings.warn(meta_warning(meta))


In [97]:
ddf.compute()

,country,result
0,Thailand,"Success({'id': 711, 'master_id': '2023THA', 't..."
1,Poland,"Success({'id': 668, 'master_id': '2023POL', 't..."
2,Singapore,"Success({'id': 693, 'master_id': '2023SGP', 't..."
3,Switzerland,"Success({'id': 708, 'master_id': '2023CHE', 't..."
4,Japan,"Success({'id': 603, 'master_id': '2023JPN', 't..."
5,India,"Success({'id': 594, 'master_id': '2023IND', 't..."
6,Kenya,"Success({'id': 606, 'master_id': '2023KEN', 't..."
7,United,"Success({'id': 657, 'master_id': '2023UMI', 't..."
8,Egypt,"Success({'id': 725, 'master_id': '2023EGY', 't..."
9,Bangladesh,"Success({'id': 507, 'master_id': '2023BGD', 't..."
